<img src="../.images/logosnb.png" alt="Banner" style="width:800px;"/>

<div style='color: #690027;' markdown="1">
    <h1>BINAIRE CLASSIFICATIE MET NEURAAL NETWERK MET EEN HIDDEN LAYER EN MET VERBETERDE SGD</h1> 
</div>

<div class="alert alert-box alert-success">
In deze notebook bouw je een neuraal netwerk op met de functionaliteiten van de Python-module Keras. Het probleem is er een van <b>binaire classificatie</b>: gevens onderverdelen in twee klassen.<br> 
Net zoals in de andere notebooks met de parabolen is de optimalizatiefunctie <em>stochastic gradient descent</em>. Maar de prestaties ervan zijn verbeterd door het toevoegen van <em><b>momentum</b></em>. Momentum zorgt ervoor dat het algoritme minder snel vast komt te zitten in een lokaal minimum. 
</div>

Voor meer uitleg over momentum verwijzen we naar de literatuur, bv. Deep Learning with Python van François Chollet [1]. 

### De nodige modules importeren

In deze notebook bouw je een *sequential model* op met Keras. Dat is een model dat bestaat uit *aaneengeschakelde lagen*. 
Je zal werken met een input layer, een output layer en ertussen één hidden layer.<br>
Je zal dus het model en de lagen moeten kunnen aanmaken.

In [ ]:
import pandas as pd

import matplotlib.pyplot as plt
import numpy as np

from keras import models
from keras import layers
from keras import optimizers

Je gaat aan de slag met 60 gegeven punten in het vlak. Sommige punten zijn blauw, andere zijn groen. De punten vertegenwoordigen twee klassen. <br> Van elk punt is de coördinaat en de kleur gegeven.<br>
Het is de bedoeling dat de groene en blauwe punten van elkaar gescheiden worden. 

<div style='color: #690027;' markdown="1">
    <h2>1. Inlezen van de data</h2> 
</div>

Lees met de module `pandas` de dataset in.

In [ ]:
punten = pd.read_csv("../.data/IntroductieDeepLearning/parabolen.dat")  # in te lezen tabel heeft geen hoofding

<div style='color: #690027;' markdown="1">
    <h2>2. Tonen van de ingelezen data</h2> 
</div>

Bekijk de data door de instructie `punten` uit te voeren. De dataset bestaat uit de x- en y-coördinaat van de punten en de kleur van elk punt. <br>
De x- en y-coördinaat zijn kenmerken, de kleur is een label. <br> Omdat er twee soorten labels zijn, zegt men dat de punten verdeeld zijn over **twee klassen**.

In [ ]:
punten

Deze tabel is een tabel met 60 rijen en 3 kolommen: er zijn immers 60 punten, 2 kenmerken en 1 label. <br><br>
De kenmerken:
- eerste kolom: x-coördinaat; 
- tweede kolom: y-coördinaat.

Het label:
- derde kolom: kleur.

<div class="alert alert-box alert-info">
In machinaal leren worden twee kenmerken doorgaans weergegeven met x1 en x2 en het label met y.
</div>

<div style='color: #690027;' markdown="1">
    <h2>3. Onderzoeken of de punten van elkaar gescheiden kunnen worden</h2> 
</div>

Of punten scheidbaar zijn, zie je het best op een grafiek. 

Om de data te visualiseren, heb je de x- en y-coördinaat, dus de kenmerken x1 en x2, van de punten nodig.<br>
Daarnaast moet je ook rekening houden met de kleur.

<div style='color: #690027;' markdown="1">
    <h3>3.1 Verschillende soorten data opslaan in verschillende NumPy arrrays</h3> 
</div>

In [ ]:
# pandas-tabel omzetten naar NumPy array
punten = np.array(punten)

# kenmerken en labels apart
X = punten[:,0:2]      # kenmerken staan in eerste twee kolommen
y = punten[:,2]        # kleur staat in laatste kolom

# coördinaten apart bekijken
x1 = X[:, 0]            # x-coördinaat staat in kolom met index 0
x2 = X[:, 1]            # y-coördinaat staat in kolom met index 1
# datapunten standaardiseren
x1 = (x1 - np.mean(x1))/np.std(x1)
x2 = (x2 - np.mean(x2))/np.std(x2)
X = np.stack((x1, x2), axis = 1)    # juiste formaat, axis=1 zet x1 en x2 als kolommen

<div style='color: #690027;' markdown="1">
    <h3>3.2 De data weergeven in puntenwolk</h3> 
</div>

In [ ]:
plt.figure()     

plt.scatter(x1[:30], x2[:30], color="blue", marker="x")    
plt.scatter(x1[30:], x2[30:], color="green", marker="<") 

plt.show()

De punten zijn niet lineair scheidbaar maar wel scheidbaar door een kromme.

<div style='color: #690027;' markdown="1">
    <h2>4. Opbouw van een neuraal netwerk</h2> 
</div>

<div style='color: #690027;' markdown="1">
    <h3>4.1 Architectuur van het neuraal netwerk</h3> 
</div>

In [ ]:
# splitsen in training en validatiedata
x1_val = np.append(x1[10:15],x1[40:45])
x2_val = np.append(x2[10:15],x2[40:45]) 
X_val = np.stack((x1_val, x2_val), axis=1)    # juiste formaat, axis=1 zet x1 en x2 als kolommen
x1_train = np.append(np.append(x1[0:10], x1[15:40]), x1[45:])
x2_train = np.append(np.append(x2[0:10], x2[15:40]), x2[45:])
X_train = np.stack((x1_train, x2_train), axis=1)  

# labels numeriek weergeven
y[y == "blauw"] = 0 
y[y == "groen"] = 1 
print(y)
# y = np.array(y)
# y = np.append(np.zeros((30,1)),np.ones((30,1)))
# y_val =  utils.to_categorical(np.append(y[0:5], y[30:35]))
# y_train = utils.to_categorical(np.append(y[5:30], y[35:]))
y_val =  np.append(y[10:15], y[40:45])
y_train = np.append(np.append(y[0:10], y[15:40]), y[45:])

In [ ]:
print(y_val)
print(y_train)

In [ ]:
plt.scatter(X_train[:,0],X_train[:,1])
plt.scatter(X_val[:,0],X_val[:,1])

In [ ]:
# architectuur netwerk
network = models.Sequential()
network.add(layers.Dense(8, activation="relu", input_dim=2))    # hidden layer: 8 neuronen, activatie ReLU      
network.add(layers.Dense(4, activation="relu"))                 # hidden layer: 4 neuronen, activatie RelU      
network.add(layers.Dense(1, activation='sigmoid'))              # output layer: 1 output neuron, activatie sigmoïde
sgd = optimizers.SGD(lr=0.1, decay=1e-6, momentum=0.9, nesterov=True)  # optimizer: SGD,met momentum voor betere prestatie  
network.compile(optimizer=sgd,
                loss="binary_crossentropy",
                metrics=["accuracy"])                            # optimizer, loss en metrics kiezen

<div style='color: #690027;' markdown="1">
    <h3>4.2 Trainen van het neuraal netwerk</h3> 
</div>

In [ ]:
# trainen van het netwerk met methode `fit`, m.a.w. beelden en labels op elkaar afstemmen
# 40 epochs
history = network.fit(X_train, y_train, epochs=40, validation_data=(X_val,y_val))   # batch_size is standaardinstelling, nl. 32
loss = history.history["loss"]
epochs = range (1 , len(loss) +1)
acc = history.history["accuracy"]
val_acc = history.history["val_accuracy"]
val_loss = history.history["val_loss"]

<div style='color: #690027;' markdown="1">
    <h3>4.3 Prestatie van het model</h3> 
</div>

In [ ]:
font = {'family': 'serif',
        'color':  'black',
        'weight': 'normal',
        'size': 14,
        }
plt.figure(figsize=(16,8))

plt.subplot(1,2,1)
plt.plot(epochs, loss, color="blue", label="train", linewidth=2)
plt.plot(epochs, val_loss, color="lightblue", label="val", linewidth=2)
plt.xticks(np.arange(0, 41, step=10))             
plt.title("Loss op training- en validatieset", fontdict=font)
plt.xlabel("epoch", fontdict=font)
plt.ylabel("loss", fontdict=font)
plt.legend(loc="lower left")

plt.subplot(1,2,2)
plt.plot(epochs, acc, color="green", label="train", linewidth=2)
plt.plot(epochs, val_acc, color="lime", label="val", linewidth=2)
plt.xticks(np.arange(0, 41, step=10)) 

plt.xlabel("epoch", fontdict=font)
plt.ylabel("acc", fontdict=font)

plt.title("Accuracy op training- en validatieset", fontdict=font)
plt.legend(loc="lower right")


plt.show()

<div style='color: #690027;' markdown="1">
    <h2>5. Decision boundary</h2> 
</div>

In [ ]:
# decision boundary
font = {'family': 'serif',
        'color':  'black',
        'weight': 'normal',
        'size': 14,
        }
color = ["blue", "green"]
soort = ["parabool 1", "parabool 2"]
                 

plt.figure(figsize=(8, 6))

for target in range(2):
    X_plot = X[y == target]
    plt.scatter(X_plot[:, 0], X_plot[:, 1], marker='o', color=color[target], label=soort[target])
plt.title("Classificatie punten twee parabolen", fontdict=font)
plt.xlabel("x-coördinaat", fontdict=font)
plt.ylabel("y-coördinaat", fontdict=font)
plt.legend(loc="upper right")

as1 = np.linspace(-2, 2, 40)
as2 = np.linspace(-2, 5, 40)

# resolutie
xx1 = np.arange(as1.min()-1, as1.max()+1, 0.1)
xx2 = np.arange(as2.min()-1, as2.max()+1, 0.1)                     

for a in xx1:
    for b in xx2:
        P = np.array([[a, b]])
        voorspeldeklasse= network.predict_classes(P) 
        if voorspeldeklasse == 0:
            kleur = "lightblue"
        else:
            kleur = "lightgreen"
        plt.plot(a, b, marker='.', color=kleur)

plt.show()

<div class="alert alert-box alert-info">
De data bestaan uit punten met twee <b>kenmerken</b> en een overeenkomstig <b>label</b>. Het label kan twee waarden aannemen; er zijn twee <b>klassen</b>. Een grens tussen de klassen is een <b>decision boundary</b>. <br>
Het model is een neuraal netwerk met een invoerlaag, een verborgen laag met activatiefunctie <b>ReLU</b> en een uitvoerlaag met activatiefunctie de <b>sigmoïde</b>-functie. De kostenfunctie is <b>binary cross-entropy</b> en de optimalisatiefunctie is <b>stochastic gradient descent met momentum</b>. <br>
</div>

<div>
    <h2>Referentielijst</h2> 
</div>

[1] Chollet, F. (2018). *Deep Learning with Python*. Manning Publications Co.

<img src="../.images/cclic.png" alt="Banner" align="left" style="width:80px;"/><br><br>
Notebook KIKS, zie <a href="http://www.aiopschool.be">AI Op School</a>, van F. wyffels & N. Gesquière is in licentie gegeven volgens een <a href="http://creativecommons.org/licenses/by-nc-sa/4.0/">Creative Commons Naamsvermelding-NietCommercieel-GelijkDelen 4.0 Internationaal-licentie</a>.